In [1]:
# Step 1: get all data
from main import setup
dataframes, effect_data = setup("", True)

### Strength Metric Calculation

To evaluate the strength of an ability, we incorporate several factors: damage, effects, cooldown, and area of effect (AoE). Each factor contributes to a final score that reflects the overall potency of the ability.

#### Variables:

- **$ \text{AvgDmg} $**: Average damage dealt by the ability (computed using dice).
- **$ \text{EffScore} $**: The total effect score, considering both the effect’s strength and duration.
- **$ \text{CDPenalty} $**: A cooldown penalty based on the cooldown duration.
- **$ \text{AoEArea} $**: The area of effect based on the type and range of the ability.
- **$ r_i $**: The rating (or strength) of an individual effect $ i $ based on a scale from 1 to 5.
- **$ d_i $**: The duration of effect $ i $ in rounds (default of 1 if not specified).

#### Formulas:

1. **Effect Score**:

   $$
   \text{EffScore} = \sum_{i} r_i \times d_i
   $$

   where:
   - $ r_i $ is the strength rating of effect $ i $.
   - $ d_i $ is the duration of effect $ i $ in rounds.

<br>

2. **Cooldown Penalty**:

   $$
   \text{CDPenalty} = \text{CD}
   $$

   where:
   - $ \text{CD} $ is the cooldown duration in rounds.

<br>

3. **Area of Effect (AoE) Calculation**:

   The calculation varies based on the type of AoE:
   - For "radius" attacks (circular area):
     $$
     \text{AoEArea} = \pi \times \left(\text{range}\right)^2
     $$
   - For "cone" attacks:
     $$
     \text{AoEArea} = 0.5 \times \left(\text{range}\right)^2 \times \left(\frac{\pi}{3}\right)
     $$
   - For "line" attacks (assuming a width of 5 feet):
     $$
     \text{AoEArea} = \text{range} \times 5
     $$
   - For "self" or "single" attacks:
     $$
     \text{AoEArea} = 1
     $$

<br>

4. **Final Strength Metric**:

   $$
   \text{Strength Metric} = \left(\text{AvgDmg} + \text{EffScore}\right) \times \sqrt{\text{AoEArea}} - \text{CDPenalty}
   $$


### Example Calculation:

For an ability with:

- **Average Damage** = 12
- **Effects**: Petrified with a rating of 5 and a duration of 3 rounds
- **Cooldown**: 3 rounds
- **AoE Type**: Radius with a range of 10 feet

1. **Effect Score**:
   $$
   \text{EffScore} = 5 \times 3 = 15
   $$

2. **Cooldown Penalty**:
   $$
   \text{CDPenalty} = 3
   $$

3. **Area of Effect**:
   $$
   \text{AoEArea} = \pi \times (10)^2 = 314.16
   $$

4. **Final Strength Metric**:
   $$
   \text{Strength Metric} = (12 + 15) \times \sqrt{314.16} - 3 = 27 \times 17.73 - 3 \approx 477.71 - 3 = 474.71
   $$

In [2]:
import math
import pandas as pd
import numpy as np
import re

# Step 2: Ability Rating
action_df = dataframes["actions"]
# Step 2.1: Calculate dice damage (average, max, min, sd)
def parse_dice(dice_str):
    """Parses dice in format 'XdY+Z', 'XdY-Z', or 'XdY' and returns a tuple (X, Y, Z)."""
    pattern = r'(\d+)d(\d+)([+-]\d+)?'
    match_obj = re.fullmatch(pattern, dice_str.replace(' ', ''))
    if match_obj:
        X = int(match_obj.group(1))
        Y = int(match_obj.group(2))
        Z = int(match_obj.group(3)) if match_obj.group(3) else 0
        return X, Y, Z
    else:
        raise ValueError(f"Invalid dice string: {dice_str}")

def parse_dice_average(dice_str):
    """Calculates the average value of a dice roll in the form XdY+Z."""
    X, Y, Z = parse_dice(dice_str)
    return (X * (Y + 1) / 2) + Z

def combined_damage_statistics(damage_entries):
    total_min_damage = 0
    total_max_damage = 0
    total_avg_damage = 0
    total_variance = 0

    for damage in damage_entries:
        if isinstance(damage, list):
            dice_str = damage[0]
        else:
            dice_str = damage
        X, Y, Z = parse_dice(dice_str)

        min_damage = X * 1 + Z
        max_damage = X * Y + Z
        avg_damage = (X * (Y + 1) / 2) + Z
        variance = X * (((Y ** 2) - 1) / 12)

        total_min_damage += min_damage
        total_max_damage += max_damage
        total_avg_damage += avg_damage
        total_variance += variance

    combined_std_dev = np.sqrt(total_variance)
    return total_min_damage, total_max_damage, total_avg_damage, combined_std_dev

# Step 2.2: Calculate the area of effect based on AoE type and range 
def calculate_aoe_area(aoe_type, range_value):
    # Convert range from feet to tiles (1 tile = 5 x 5 feet)
    range_in_tiles = int(int(range_value) / 5) if range_value else 0
    match aoe_type:
        case "radius":
            area = round(math.pi * (range_in_tiles ** 2))
        case "cone":
            area = range_in_tiles ** 2
        case "line":
            area = range_in_tiles
        case "self":
            area = 1
        case _:
            area = 1  # Assume single-target has minimal area impact
    return area

# Step 2.3: Calculate effect score based on ratings and parameter
def get_multiplier(parameter):
    if isinstance(parameter, (int, float)):
        print(f"parameter {parameter} is an integer")
        return parameter
    elif isinstance(parameter, str):
        duration = parameter.strip()
        match duration:
            case d if d.endswith('r'):
                return int(d.rstrip('r'))
            case d if d.startswith('lv'):
                return int(d.lstrip('lv'))
            case "until freed":
                return 1  # Special case
            case d if re.fullmatch(r'\d+d\d+([+-]\d+)?', d):
                return parse_dice_average(d)
            case d if d.endswith('ft'):
                return int(d.rstrip('ft'))
            case _:
                return 1  # Default multiplier if parsing fails
    else:
        return 1  # Default multiplier

def calculate_effect_value(effect_name, rating, parameter):
    """Calculates the effect value based on effect name, rating, and duration."""
    multiplier = get_multiplier(parameter)
    match effect_name:
        case "heal":
            effect_value = rating + multiplier
            print(f"Calculating 'heal' effect: rating {rating} + average heal {multiplier} = {effect_value}")
            return effect_value
        case "grappled":
            # TODO adjust the special case
            if parameter == "until freed":
                effect_value = rating * 2
                print(f"Calculating 'grappled' effect with 'until freed': rating {rating} * 2 = {effect_value}")
            else:
                effect_value = rating * multiplier
                print(f"Calculating 'grappled' effect: rating {rating} * duration multiplier {multiplier} = {effect_value}")
            return effect_value
        case _:
            # Default calculation
            # for effects with range 
            if parameter.endswith('ft'):
                effect_value = rating + multiplier/5 # distance converted in tiles
                print(f"Calculating default effect '{effect_name}': rating {rating} + distance {multiplier} = {effect_value}")
            # for the rest    
            else:    
                effect_value = rating * multiplier
                print(f"Calculating default effect '{effect_name}': rating {rating} * multiplier {multiplier} = {effect_value}")
            return effect_value

def calculate_effect_score(effects):
    effect_score = 0
    for effect in effects:
        effect_name = effect[0]
        duration = effect[1] if len(effect) > 1 else 1
        rating = effect_data.get(effect_name, 0)  # Default to 0 if the effect is not found
        effect_value = calculate_effect_value(effect_name, rating, duration)
        effect_score += effect_value
    return effect_score

# Step 2.4: Calculate a penalty for cooldowns
def calculate_cooldown_penalty(cooldown):
    if isinstance(cooldown, list) and len(cooldown) > 0:
        penalty = int(cooldown[0])
    else:
        penalty = 0
    return penalty

# Step 2.5: Calculate a final metric based on all factors
def calculate_strength_metric(row):
    damages = row['damage']
    min_dmg, max_dmg, avg_dmg, std_dmg = combined_damage_statistics(damages)
    damage_score = avg_dmg
    effect_score = calculate_effect_score(row['effects'])
    cooldown_penalty = calculate_cooldown_penalty(row['cooldown'])
    aoe_area = calculate_aoe_area(row['aoe'], row['range'])
    # TODO Adjust the penalty for cooldowns as needed
    strength_metric = (damage_score + effect_score) * (aoe_area ** 0.5) - cooldown_penalty
    return strength_metric


action_df['strength_metric'] = action_df.apply(calculate_strength_metric, axis=1)
action_df


Calculating default effect 'exhausted': rating 5 * multiplier 4 = 20
Calculating default effect 'prone': rating 3 * multiplier 1 = 3
Calculating default effect 'blinded': rating 3 * multiplier 1 = 3
Calculating default effect 'paralysed': rating 5 * multiplier 2 = 10
Calculating default effect 'bleeding': rating 5 * multiplier 1 = 5
Calculating default effect 'bleeding': rating 5 * multiplier 2 = 10
Calculating default effect 'bleeding': rating 5 * multiplier 2 = 10
Calculating default effect 'petrified': rating 5 * multiplier 3 = 15
Calculating default effect 'difficult_terrain': rating 2 * multiplier 20 = 40
Calculating default effect 'resistance_(all)': rating 4 * multiplier 1 = 4
Calculating default effect 'bleeding': rating 5 * multiplier 1 = 5
Calculating default effect 'petrified': rating 5 * multiplier 3 = 15
Calculating default effect 'escape': rating 3 + distance 30 = 9.0
Calculating default effect 'knockback': rating 3 + distance 20 = 7.0
Calculating default effect 'knockbac

,name,legendary,cooldown,range,aoe,hitbonus,savereq,damage,effects,strength_metric
0,Abyssaler Nebelhauch,False,"[3, r]",70,radius,0,"[con, 19]","[[12d6, acid]]","[[exhausted, lv4]]",1535.799532
1,Beilhieb,False,[],5,single,5,[],"[[2d8, slashing]]",[],9.000000
2,Actionsname,False,"[1, d]",40,line,9,[],"[[3d8+6, slashing]]","[[prone, 1r]]",62.639610
3,Blendgranate,False,"[1, d]",30,radius,0,"[con, 16]",[],"[[blinded, 1r]]",30.890437
4,Blitz aus der Tiefe,False,"[3, r]",100,single,0,"[dex, 18]","[[2d10, lightning]]","[[paralysed, 2r]]",18.000000
5,Blutige Rache,False,[],5,single,10,[],"[[2d10+6, slashing], [2d8, necrotic]]","[[bleeding, lv1]]",31.000000
6,Blutiger Schlag,False,[],5,single,10,[],"[[2d10+6, slashing], [2d10+6, slashing]]",[],34.000000
7,Blutwoge,False,"[3, r]",15,cone,0,"[dex, 17]","[[5d8, necrotic], [3d6, slashing]]","[[bleeding, lv2]]",126.000000
8,Bohrschnabel,False,[],5,single,5,[],"[[1d6+2, piercing]]",[],5.500000
9,Brutale Hiebe,False,[],5,single,10,[],"[[2d12+6, slashing], [2d12+6, slashing]]","[[bleeding, lv2]]",48.000000


### Effect Value Calculation

Calculate the **effect value** based on the **effect name**:

1. **For 'heal':**

   $$
   \text{effect\_value} = \text{rating}(\text{'heal'}) + \text{average dice roll}
   $$

2. **For 'grappled':**

   - **If duration is 'until freed':**

     $$
     \text{effect\_value} = \text{rating}(\text{'grappled'}) \times 2
     $$

   - **Else:**

     $$
     \text{effect\_value} = \text{rating}(\text{'grappled'}) \times \text{duration}
     $$

3. **For other effects:**

   - **If duration ends with 'ft':**

     $$
     \text{effect\_value} = \text{rating}(\text{effect\_name}) + \frac{\text{distance}}{5}
     $$

     (Adds distance in tiles to the rating)

   - **Else:**

     $$
     \text{effect\_value} = \text{rating}(\text{effect\_name}) \times \text{duration or level}
     $$

In [ ]:
# Step 3: calculate strength of enemy